In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import keras as K

## 1-Layer Neural Network (Multi-Layer-Perceptron)
---

In [ ]:
#--- Create a complicated dataset ---#
from sklearn.datasets import make_moons
(X,y) = make_moons(n_samples=500, shuffle=True, noise=0.1)

#--- Visualize ---#
plt.figure(figsize=(10,10))
plt.plot(X[y==0,0],X[y==0,1],'.b')
plt.plot(X[y==1,0],X[y==1,1],'.r')

In [ ]:
#--- Perceptron Parameters ---#
perceptron_NHidden = 2000
batchSize = 128
epochs=500
nonlinearity = 'tanh'

#--- Create a Perceptron ---#
from keras.models import Sequential
from keras.layers import Dense, Activation

# Add a Dense layer and an Output layer
snn = Sequential()
snn.add(Dense(perceptron_NHidden, input_dim=2, activation=nonlinearity))
snn.add(Dense(1, activation='sigmoid'))

# Decide on the metrics for the model
snn.compile(optimizer='rmsprop',
              loss='mean_squared_error',
              metrics=['accuracy'])

# Fit to the training data
p_history = snn.fit(X, y, epochs=epochs, batch_size=batchSize, verbose=0)

In [ ]:
#--- Visualize Decision Boundary ---#
n = 250
hdom = np.linspace(-2,3,n) 
vdom = np.linspace(-2,3,n) 
HH, VV  = np.meshgrid(hdom,vdom)

XPred = np.c_[HH.ravel(), VV.ravel()]
yPred = snn.predict(XPred)
yTrain = snn.predict(X).ravel()
yTrainC = yTrain > 0.5

yPred = np.reshape(yPred,(n,n))

# Prediction Figure
plt.figure(figsize=(15,5))
plt.subplot(131)
plt.imshow(yPred, cmap='RdBu_r', origin='lower', extent=(-2, 3, -2, 3))
plt.plot(X[y==0,0],X[y==0,1],'ob',alpha=0.5)
plt.plot(X[y==1,0],X[y==1,1],'or', alpha=0.5)
plt.title('Prediction', fontsize=18)

# Decision Boundary Figure
plt.subplot(132)
plt.imshow(yPred > 0.5, cmap='bwr', origin='lower', extent=(-2, 3, -2, 3))
plt.plot(X[np.not_equal(y,yTrainC),0],X[np.not_equal(y,yTrainC),1],'xk', markeredgecolor='k')
plt.title('Decision Boundary', fontsize=18)

# Final Training Prediction
plt.subplot(133)
plt.scatter(X[:,0],X[:,1],c=yTrain, cmap='bwr')
plt.title('Training Prediction', fontsize=18)

plt.tight_layout()

#--- Display Final Test Error ---#
training_error = np.mean(np.not_equal(y,yTrainC))
print('--------\nTraining Error = %0.2f%%\n--------' % (training_error*100))

## Neural Network with Depth
----

In [ ]:
#--- MLP Parameters ---#
mlp_NHidden = (10,10,10,10)
batchSize = 32
epochs=500
layers = len(mlp_NHidden)
nonlinearity = 'tanh'

#--- Create a Perceptron ---#
# Add a Dense layer and an Output layer
MLP = Sequential()

# Add the first layer
MLP.add(Dense(mlp_NHidden[0], input_dim=2, activation=nonlinearity))

# Add remaining intermediate layers
for l in range(1,layers):
    MLP.add(Dense(mlp_NHidden[l], activation=nonlinearity))

# Add final layer
MLP.add(Dense(1, activation='sigmoid'))

# Decide on the metrics for the model
MLP.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Fit to the training data
mlp_history = MLP.fit(X, y, epochs=epochs, batch_size=batchSize)

In [ ]:
#--- Visualize Decision Boundary ---#
n = 250
hdom = np.linspace(-2,3,n) 
vdom = np.linspace(-2,3,n) 
HH, VV  = np.meshgrid(hdom,vdom)

XPred = np.c_[HH.ravel(), VV.ravel()]
yPred = MLP.predict(XPred)
yTrain = MLP.predict(X).ravel()
yTrainC = yTrain > 0.5

yPred = np.reshape(yPred,(n,n))

# Prediction Figure
plt.figure(figsize=(15,5))
plt.subplot(131)
plt.imshow(yPred, cmap='RdBu_r', origin='lower', extent=(-2, 3, -2, 3))
plt.plot(X[y==0,0],X[y==0,1],'ob',alpha=0.5)
plt.plot(X[y==1,0],X[y==1,1],'or', alpha=0.5)
plt.title('Prediction', fontsize=18)

# Decision Boundary Figure
plt.subplot(132)
plt.imshow(yPred > 0.5, cmap='bwr', origin='lower', extent=(-2, 3, -2, 3))
plt.plot(X[np.not_equal(y,yTrainC),0],X[np.not_equal(y,yTrainC),1],'xk', markeredgecolor='k')
plt.title('Decision Boundary', fontsize=18)

# Final Training Prediction
plt.subplot(133)
plt.scatter(X[:,0],X[:,1],c=yTrain, cmap='bwr')
plt.title('Training Prediction', fontsize=18)

plt.tight_layout()

#--- Display Final Test Error ---#
training_error = np.mean(np.not_equal(y,yTrainC))
print('--------\nTraining Error = %0.2f%%\n--------' % (training_error*100))

In [ ]:
#--- Comparison ---#
perceptron_nparams = 2*perceptron_NHidden + perceptron_NHidden + perceptron_NHidden + 1
mlp_nparams = 0
for layer in MLP.layers:
    # Sum together  weights------------------------      and biases ------------------------
    #                         /                    \                    /                   \
    mlp_nparams += np.asarray(layer.get_weights()[0]).size + np.asarray(layer.get_weights()[1]).size
    


p_training_err = 1 - np.asarray(p_history.history['acc'])
mlp_training_err = 1 - np.asarray(mlp_history.history['acc'])

p_training_nacc = np.asarray(p_history.history['acc']) / perceptron_nparams
mlp_training_nacc = np.asarray(mlp_history.history['acc']) / mlp_nparams

plt.figure(figsize=(10,10))
plt.subplot(211)
plt.plot(p_training_err,   label='Perceptron')
plt.plot(mlp_training_err, label='MLP')
plt.xlabel('Training Epochs', fontsize=16)
plt.ylabel('Training Error', fontsize=16)
plt.xlim((0,epochs-1))
plt.legend(loc=1, fontsize=16)

plt.subplot(212)
plt.plot(p_training_nacc,   label='Perceptron')
plt.plot(mlp_training_nacc, label='MLP')
plt.xlabel('Training Epochs', fontsize=16)
plt.ylabel('Per-Model-Parameter Accuracy', fontsize=16)
plt.xlim((0,epochs-1))
plt.legend(loc=4, fontsize=16)

plt.tight_layout()

print('1-Layer # Parmeters:    ', perceptron_nparams)
print('Deep Model # Parmeters: ', mlp_nparams)

## Auto-Encoder
---

In [ ]:
#--- Auto Encoder Parameters ---#
batchSize = 128
epochs = 1024

#--- Create a Perceptron ---#
# Add a Dense layer and an Output layer
MLP = Sequential()


MLP.add(Dense(60, input_dim=2, activation=nonlinearity))

MLP.add(Dense(20, activation='relu'))

MLP.add(Dense(1, activation='relu'))

MLP.add(Dense(20, activation='relu'))

MLP.add(Dense(60, activation='relu'))

# Add final layer
MLP.add(Dense(2, activation='linear'))

# Decide on the metrics for the model
MLP.compile(optimizer='rmsprop',
              loss='mean_squared_error',
              metrics=['mse'])

# Fit to the training data
X = X[np.random.permutation(X.shape[0]),:]
mlp_history = MLP.fit(X, X, epochs=epochs, batch_size=batchSize)

In [ ]:
Xout = MLP.predict(X)

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(X[:,0],X[:,1],'.b')
plt.plot(Xout[:,0],Xout[:,1],'or')

In [ ]:
decoder = Sequential()
decoder.add(Dense(20, input_dim=1, activation='relu'))
decoder.add(Dense(60, activation='relu'))
decoder.add(Dense(2, activation='linear'))

decoder.layers[0].set_weights(MLP.layers[3].get_weights())
decoder.layers[1].set_weights(MLP.layers[4].get_weights())
decoder.layers[2].set_weights(MLP.layers[5].get_weights())

In [ ]:
Z = decoder.predict(np.linspace(-5,5,400))

plt.figure(figsize=(10,10))
plt.plot(X[:,0],X[:,1],'.b')
plt.plot(Z[:,0], Z[:,1], '-', linewidth=3)

In [ ]:
Z.shape